In [1]:
import bs4
import torch
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_huggingface.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, pipeline)
from typing_extensions import List, TypedDict

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
DEVICE = ('cuda' if torch.cuda.is_available() else
          'mps' if torch.backends.mps.is_available() else 'cpu')
DEVICE

'mps'

In [3]:
embeddings = HuggingFaceEmbeddings(model_name='all-mpnet-base-v2',
                                   model_kwargs={'device': DEVICE})

In [4]:
len(embeddings.embed_documents(['test'])[0])

768

In [5]:
index_name = 'tutorial'

pc = Pinecone()
# pc.delete_index(index_name)
# pc.create_index(
#     name=index_name,
#     dimension=768,
#     metric='cosine',
#     spec=ServerlessSpec(cloud="aws", region="us-east-1")
# )
index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [ ]:
urls = ["https://www.apple.com/"]
loader = WebBaseLoader(urls)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

# Index chunks
_ = vector_store.add_documents(documents=docs)

In [6]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)

# Define the user's question
query = "What new products are announced on Apple.com?"

# Retrieve relevant documents based on the query
relevant_docs = retriever.invoke(query)

In [7]:
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")


--- Relevant Documents ---
Document 1:
Apple


















Apple

AppleStoreMaciPadiPhoneWatch
VisionAirPodsTV & HomeEntertainmentAccessoriesSupport


0+







 


MacBook Air
Sky blue color.Sky high performance with M4.

Learn more
Buy



Built for Apple Intelligence.












 


iPad Air
Now supercharged by the M3 chip.

Learn more
Buy



Built for Apple Intelligence.












 


Mac Studio
M4 Max and M3 Ultra. Choose your superpower.

Learn more
Buy



Built for Apple Intelligence.














 

iPad
Now with the speed of the A16 chip and double the starting storage.

Learn more
Buy












 

iPhone
Meet the iPhone 16 family.

Learn more
Shop iPhone

Built for Apple Intelligence.











 

Apple Trade In
Get $170–$630 in credit when you trade in iPhone 12 or higher.1

Get your estimate












 

Apple Watch Series 10
Thinstant classic.

Learn more
Buy












 

AirPods Pro 2
Now with a Hearing Aid feature.2


Learn more
Buy

Source: https://www.ap

In [8]:
model_name = "HuggingFaceH4/zephyr-7b-beta"
model = AutoModelForCausalLM.from_pretrained(model_name)
model

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

LlamaTokenizerFast(name_or_path='HuggingFaceH4/zephyr-7b-beta', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='left', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>', 'additional_special_tokens': ['<unk>', '<s>', '</s>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [11]:
llm = pipeline(task='text-generation', model=model_name, device=DEVICE)
llm

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use mps


In [12]:
llm("What is 4+4? Answer:")

KeyboardInterrupt: 

In [ ]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [ ]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()